In [8]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import requests
from datetime import datetime
from bs4 import BeautifulSoup
import requests
import logging

In [13]:
data = pd.read_csv('../data/vacancies_ru.csv')
data.drop('Unnamed: 0', axis=1, inplace=True)
data

,vacancy_id,name,company_id,keySkills,compensation_from,compensation_to,compensation_currencyCode,area_id,area_regionId,employment,workSchedule,workExperience,clean_name,area_regionId_encoder,area_id_encoder,company_id_encoder,employment_encoder,workSchedule_encoder,workExperience_encoder
0,v_862116,Смотритель музейный,c_162972,"['Пользователь ПК', 'Работа в команде', 'Умени...",16500.0,NaN,RUR,a_4761,ar_33,full,fullDay,noExperience,Смотритель музейный,32,4181,69972,5,5,1
1,v_288642,Ведущий менеджер по работе с физическими лицами,c_208672,"['Активные продажи', 'Холодные продажи', 'Кред...",50000.0,NaN,RUR,a_744,ar_2,full,fullDay,noExperience,Ведущий менеджер по работе с физическими лицами,17,6731,120750,5,5,1
2,v_1840054,Бухгалтер (по расчету зарплаты),c_198109,NaN,50000.0,65000.0,RUR,a_6223,ar_78,full,fullDay,between3And6,Бухгалтер,81,5806,109013,5,5,3
3,v_2346232,"Пекарь (Токсово, Привокзальная, 16)",c_6137,NaN,38500.0,42000.0,RUR,a_4795,ar_51,full,fullDay,noExperience,Пекарь,52,4218,235992,5,5,1
4,v_312507,Торговый представитель (г. Абакан),c_206699,"['Продуктивность', 'Клиентоориентированность',...",60000.0,NaN,RUR,a_6837,ar_4,full,fullDay,between1And3,Торговый представитель,39,6487,118557,5,5,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2209567,v_2484959,Сборщик-упаковщик,c_203256,"['Пользователь ПК', 'Умение работать в команде...",40000.0,90000.0,RUR,a_5387,ar_71,full,flexible,noExperience,Сборщик-упаковщик,74,4876,114733,5,2,1
2209568,v_205163,Сварщик на полуавтомат,c_158695,"['Желание работать и зарабатывать', 'Высокая э...",80000.0,130000.0,RUR,a_5527,ar_69,full,fullDay,between1And3,Сварщик на полуавтомат,71,5032,65219,5,5,2
2209569,v_639897,Главный инженер / Технический директор,c_209365,"['Контроль исправности оборудования', 'Инженер...",200000.0,NaN,RUR,a_1756,ar_41,full,fullDay,between1And3,Главный инженер,41,842,121520,5,5,2
2209570,v_1636531,"Провизор/Фармацевт (г.Адыгейск, 20 км от Красн...",c_246244,"['Предпечатная подготовка', 'Статистический ан...",25000.0,NaN,RUR,a_3403,ar_60,full,fullDay,noExperience,Фармацевт,62,2673,162497,5,5,1


In [14]:
data['workSchedule'].unique()

array(['fullDay', 'flyInFlyOut', 'flexible', 'shift', 'remote'],
      dtype=object)

In [21]:
data[data['workSchedule'] == 'flyInFlyOut'].head()

,vacancy_id,name,company_id,keySkills,compensation_from,compensation_to,compensation_currencyCode,area_id,area_regionId,employment,workSchedule,workExperience,clean_name,area_regionId_encoder,area_id_encoder,company_id_encoder,employment_encoder,workSchedule_encoder,workExperience_encoder
7,v_2091086,Монтажник СТиЖБК,c_230890,NaN,100000.0,NaN,RUR,a_500,ar_36,full,flyInFlyOut,between1And3,Монтажник СТиЖБК,35,4447,145437,5,1,2
23,v_2416809,Водитель автомобиля 6-го разряда (БЕЛАЗ -7555),c_220194,"['Умение работать в команде', 'Точность и вним...",138100.0,138100.0,RUR,a_3095,ar_20,full,flyInFlyOut,between3And6,Водитель автомобиля 6-го разряда,18,2330,133553,5,1,3
60,v_1070312,Водитель самосвала,c_18924,"['Водительское удостоверение категории BC', 'З...",145000.0,180000.0,RUR,a_4551,ar_28,full,flyInFlyOut,between1And3,Водитель самосвала,26,3948,99158,5,1,2
73,v_194247,Механик бурового оборудования,c_109998,"['Техническое обслуживание', 'Точность и внима...",250000.0,NaN,RUR,a_3445,ar_56,full,flyInFlyOut,between1And3,Механик бурового оборудования,57,2719,11111,5,1,2
77,v_1685474,Машинист бульдозера SHANTUI SD-16,c_13187,NaN,160000.0,NaN,RUR,a_500,ar_36,full,flyInFlyOut,between1And3,Машинист бульдозера SHANTUI SD-16,35,4447,35414,5,1,2


In [20]:
data[data['workSchedule'] == 'shift'].head()

,vacancy_id,name,company_id,keySkills,compensation_from,compensation_to,compensation_currencyCode,area_id,area_regionId,employment,workSchedule,workExperience,clean_name,area_regionId_encoder,area_id_encoder,company_id_encoder,employment_encoder,workSchedule_encoder,workExperience_encoder
27,v_484279,Кладовщик на склад (Артем),c_243786,"['Приемка груза', 'Документальное сопровождени...",70000.0,NaN,RUR,a_4931,ar_23,full,shift,between1And3,Кладовщик на склад,21,4370,159765,5,4,2
29,v_2226044,Специалист по вооруженному сопровождению грузов,c_49304,NaN,70000.0,NaN,RUR,a_1756,ar_41,full,shift,between1And3,Специалист по вооруженному сопровождению грузов,41,842,222586,5,4,2
30,v_2515950,Администратор отеля,c_170361,NaN,32000.0,35000.0,RUR,a_6223,ar_78,part,shift,noExperience,Администратор отеля,81,5806,78183,4,4,1
31,v_1254531,Мастер цеха по производству ЛКМ,c_91131,"['Пользователь ПК', 'Работа в команде', 'Контр...",70000.0,NaN,RUR,a_5527,ar_69,full,shift,between3And6,Мастер цеха по производству ЛКМ,71,5032,269061,5,4,3
36,v_1545695,Комплектовщик / сборщик заказов на склад обуви...,c_235224,NaN,47000.0,52000.0,RUR,a_5111,ar_64,full,shift,between1And3,Комплектовщик,66,4571,150253,5,4,2


In [25]:
data[data['workSchedule'] == 'remote'].head()

,vacancy_id,name,company_id,keySkills,compensation_from,compensation_to,compensation_currencyCode,area_id,area_regionId,employment,workSchedule,workExperience,clean_name,area_regionId_encoder,area_id_encoder,company_id_encoder,employment_encoder,workSchedule_encoder,workExperience_encoder
32,v_1284861,Менеджер по развитию В2В,c_160874,"['Грамотная речь', 'Деловая коммуникация', 'На...",42000.0,NaN,RUR,a_1756,ar_41,full,remote,between1And3,Менеджер по развитию В2В,41,842,67641,5,3,2
35,v_2494877,Ассистент отдела продаж,c_94905,"['Работа в команде', 'Федеральные сети', 'Знан...",30000.0,30000.0,RUR,a_1756,ar_41,full,remote,between1And3,Ассистент отдела продаж,41,842,273253,5,3,2
37,v_2230711,Преподаватель чешского языка,c_13772,"['Преподаватель', 'Грамотная речь']",40000.0,NaN,RUR,a_6223,ar_78,full,remote,between1And3,Преподаватель чешского языка,81,5806,41914,5,3,2
54,v_2491017,Специалист отдела телемаркетинга (Удаленно),c_233435,"['Активные продажи', 'Работа с базами данных',...",30000.0,50000.0,RUR,a_3405,ar_81,full,remote,between1And3,Специалист отдела телемаркетинга,85,2675,148265,5,3,2
58,v_1494003,Консультант телефонной поддержки в Яндекс Прак...,c_32656,NaN,30000.0,42000.0,RUR,a_2431,ar_64,full,remote,noExperience,Консультант телефонной поддержки в Яндекс Прак...,66,1593,204088,5,3,1


In [6]:
columns = ['vacancy_id', 'name', 'company_id', 'keySkills', 'compensation_from',
           'compensation_to', 'compensation_currencyCode', 'area_id', 'employment',
           'workSchedule', 'workExperience', 'description', 'published_at']

In [2]:
url = 'https://api.superjob.ru/2.0/vacancies/'
API_KEY = 'v3.r.138336259.24bd22e30244a5ff349f72abfd00a5348a853998.b32035a7c7740e65e7ab2fcec46f96620e0005b7'

In [9]:
root = logging.getLogger()
for handler in root.handlers[:]:
    root.removeHandler(handler)

In [10]:
logging.basicConfig(
    filename='hh_logs.log',
    level=logging.INFO,
    format = '%(asctime)s - %(levelname)s - %(message)s'
    )

In [ ]:
# тут надо будет смапить с hh еще

# Список возможных значений experience:
#   1 — без опыта
#   2 — от 1 года
#   3 — от 3 лет
#   4 — от 6 лет

In [11]:
headers = {'X-Api-App-Id': API_KEY}
params = {
    'keyword': 'python',
    'page': 1,
    'count': 10
}

In [18]:
def get_employment(x):
    if x == 6: # полный день
        return 'full'
    elif x == 10 or x == 13: #неполный день или частичная занятость
        return 'part'
    elif x == 12 or x == 9: # сменный график или вахтовый метод
        return 'full'
    elif x == 7: # временная работа
        return 'project'
    else:
        return 'full'

In [39]:
def get_workSchedule(schedule_x, place_x):
    if place_x == 2:
        return 'remote'
    elif schedule_x == 6 or schedule_x == 7:
        return 'fullDay'
    elif schedule_x == 10 or schedule_x == 13:
        return 'flexible'
    elif schedule_x == 12:
        return 'shift'
    elif schedule_x == 9:
        return 'flyInFlyOut'
    else:
        return 'fullDay'

In [40]:
df = pd.DataFrame(columns=columns)

response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    
    vacancies = response.json()['objects']
    
    for i in range(len(vacancies)):
        vac = vacancies[i]

        row = {column: None for column in columns}
    
        row['vacancy_id'] = vac.get('id')
        row['name'] = vac.get('profession')
        row['company_id'] = vac.get('id_client')
        row['keySkills'] = BeautifulSoup(vac.get('candidat'), 'html.parser').get_text() # надо распарсить требования
        row['compensation_from'] = vac.get('payment_from')
        row['compensation_to'] = vac.get('payment_to')
        row['compensation_currencyCode']= vac.get('currency')
        row['area_id'] = vac.get('town')['id'] # city
        row['employment'] = get_employment(vac['type_of_work'].get('id')) if vac.get('type_of_work') else None
        row['workSchedule'] = get_workSchedule(vac['type_of_work'].get('id'), vac['place_of_work'].get('id'))\
                                                        if vac['type_of_work'] and vac['place_of_work'] else None
        row['workExperience'] = vac.get('experience').get('id') if vac.get('experience') else None
        row['description'] = BeautifulSoup(vac.get('vacancyRichText'), 'html.parser').get_text()
        row['published_at'] = datetime.fromtimestamp(vac.get('date_published')).strftime('%Y-%m-%d %H:%M:%S')
        row['alternate_url'] = vac.get('link')
        
        df = pd.concat([df, pd.DataFrame([row])], ignore_index=True)
        logging.info(f'Successful concatenation. Shape of dataframe: {df.shape}')
    
else:
    logging.info(f'Error {response.status_code}')

In [41]:
df.head()

,vacancy_id,name,company_id,keySkills,compensation_from,compensation_to,compensation_currencyCode,area_id,employment,workSchedule,workExperience,description,published_at,alternate_url
0,48945001,Преподаватель на курс “Майнкрафт на Python,2947041,"Онлайн-преподаватель курса ""Майнкрафт на Pytho...",20000,40000,rub,401,full,remote,2,"Онлайн-преподаватель курса ""Майнкрафт на Pytho...",2024-05-05 16:55:57,https://penza.superjob.ru/vakansii/prepodavate...
1,48945002,Преподаватель на курс “Майнкрафт на Python,2947041,"Онлайн-преподаватель курса ""Майнкрафт на Pytho...",20000,40000,rub,119,full,remote,2,"Онлайн-преподаватель курса ""Майнкрафт на Pytho...",2024-05-05 16:55:57,https://perm.superjob.ru/vakansii/prepodavatel...
2,48945003,Преподаватель на курс “Майнкрафт на Python,2947041,"Онлайн-преподаватель курса ""Майнкрафт на Pytho...",20000,40000,rub,73,full,remote,2,"Онлайн-преподаватель курса ""Майнкрафт на Pytho...",2024-05-05 16:55:57,https://rnd.superjob.ru/vakansii/prepodavatel-...
3,48945004,Преподаватель на курс “Майнкрафт на Python,2947041,"Онлайн-преподаватель курса ""Майнкрафт на Pytho...",20000,40000,rub,217,full,remote,2,"Онлайн-преподаватель курса ""Майнкрафт на Pytho...",2024-05-05 16:55:57,https://ryazan.superjob.ru/vakansii/prepodavat...
4,48945006,Преподаватель на курс “Майнкрафт на Python,2947041,"Онлайн-преподаватель курса ""Майнкрафт на Pytho...",20000,40000,rub,5,full,remote,2,"Онлайн-преподаватель курса ""Майнкрафт на Pytho...",2024-05-05 16:55:57,https://samara.superjob.ru/vakansii/prepodavat...


In [38]:
# draft to check fields

response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    
    vac = response.json()['objects']
    
vac[0]

{'canEdit': False,
 'is_closed': False,
 'id': 48945001,
 'id_client': 2947041,
 'payment_from': 20000,
 'payment_to': 40000,
 'date_pub_to': 1716299757,
 'date_archived': 0,
 'date_published': 1714917357,
 'address': 'Москва, Нижний Сусальный переулок, 5с19',
 'profession': 'Преподаватель на курс “Майнкрафт на Python',
 'work': None,
 'compensation': None,
 'candidat': 'Онлайн-преподаватель курса "Майнкрафт на Python" для детской школы программирования Coddy\nCODDY - это прогрессивная образовательная компания, воплощающая миссию обучения детей и подростков техническим и творческим дисциплинам, таким как программирование, дизайн и многое другое. Наша цель -помочь каждому ребенку раскрыть свой потенциал, развить креативное мышление и освоить навыки, которые будут полезны в будущем.\nПривет! Мы — Coddy, школа программирования, которая делает обучение программированию интересным и доступным для детей. Сейчас мы в поиске талантливого и креативного онлайн-преподавателя для нашего уникальног